In [1]:
import torch
from torchmetrics import AUROC
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
import os
import cv2

In [2]:
current_file_dir = os.getcwd()
train_results_json="../out_evaluate_patch_repasoD/train_scores.json"
val_results_json="../out_evaluate_patch_repasoD/val_scores.json"
test_results_json="../out_evaluate_patch_repasoD/test_scores.json"

train_results_json=os.path.join(current_file_dir,train_results_json)    
val_results_json=os.path.join(current_file_dir,val_results_json)    
test_results_json=os.path.join(current_file_dir,test_results_json)    




def create_tables(train_data,clases):
    labels=[]
    preds=[]
    filenames=[]
    for d in train_data:
        truth=[]
        pred=[]
        gt=d['ground_truth']
        scores=d['scores']
        d['filename']
        for defecto in clases:
            truth.append(gt[defecto])
            pred.append(scores[defecto])
        labels.append(truth)
        preds.append(pred)
        filenames.append(d['filename'])
            
    labels=torch.tensor(labels)
    preds=torch.tensor(preds)

    return preds,labels,filenames


with open(train_results_json, "r") as f:
    train_dict = json.load(f)

with open(val_results_json, "r") as f:
    val_dict = json.load(f) 

with open(test_results_json, "r") as f:
    test_dict = json.load(f)           

train_data=train_dict['train_results']
val_data = val_dict['val_results']
test_data = test_dict['test_results']
clases=list(train_dict['train_results'][0]['ground_truth'].keys())

print(len(train_data))
print(len(val_data))
print(len(test_data))

1138
326
162


In [3]:
preds_train,labels_train,filenames_train=create_tables(train_data,clases)
preds_val,labels_val,filenames_val=create_tables(val_data,clases)
preds_test,labels_test,filenames_test=create_tables(test_data,clases)
print(len(filenames_train))
print(len(filenames_val))
print(len(filenames_test))

1138
326
162


# Cantidad de muestras de con cada defecto

In [4]:
print("==========================================")
print("  ** TEST **")
print("==========================================")

n_por_defecto=labels_test.sum(dim=0)
for k in range(len(clases)):
    print(f'{clases[k]} : {int(n_por_defecto[k].item())}')

k=labels_test.sum(dim=1)

nbuenos=torch.sum(k==0)
print("N sin defectos:",nbuenos.item())
print("N total:",labels_test.shape[0])


print("==========================================")
print("  ** VAL **")
print("==========================================")

n_por_defecto=labels_val.sum(dim=0)
for k in range(len(clases)):
    print(f'{clases[k]} : {int(n_por_defecto[k].item())}')

k=labels_val.sum(dim=1)

nbuenos=torch.sum(k==0)
print("N sin defectos:",nbuenos.item())
print("N total:",labels_val.shape[0])

print("\n==========================================")
print("  ** TRAIN **")
print("==========================================")

n_por_defecto=labels_train.sum(dim=0)
for k in range(len(clases)):
    print(f'{clases[k]} : {int(n_por_defecto[k].item())}')

k=labels_train.sum(dim=1)

nbuenos=torch.sum(k==0)
print("N sin defectos:",nbuenos.item())
print("N total:",labels_train.shape[0])

  ** TEST **
cracks : 3
damage_by_pliers : 0
deformed_peduncle : 17
green : 20
hollow : 12
insect : 2
light_scars : 38
mechanical_damage : 7
oil_spots : 14
plu_stickers : 10
reddish : 7
rotten : 18
scars : 25
skin_breakdown : 21
surface : 39
N sin defectos: 28
N total: 162
  ** VAL **
cracks : 13
damage_by_pliers : 15
deformed_peduncle : 27
green : 30
hollow : 14
insect : 5
light_scars : 49
mechanical_damage : 14
oil_spots : 25
plu_stickers : 20
reddish : 10
rotten : 26
scars : 52
skin_breakdown : 30
surface : 78
N sin defectos: 80
N total: 326

  ** TRAIN **
cracks : 40
damage_by_pliers : 64
deformed_peduncle : 122
green : 98
hollow : 36
insect : 13
light_scars : 189
mechanical_damage : 50
oil_spots : 62
plu_stickers : 77
reddish : 33
rotten : 101
scars : 147
skin_breakdown : 137
surface : 234
N sin defectos: 263
N total: 1138


# Generar primera segunda y tercera

In [ ]:
#tercera=['cracks','rotten','skin_breakdown', 'mechanical_damage', 'damage_by_pliers', 'surface', 'insect', 'scars', 'hollow', 'skin_creasing']
tercera=['cracks','rotten','skin_breakdown', 'mechanical_damage', 'damage_by_pliers', 'surface', 'insect',  'hollow']

#segunda=['oil_spots','scars','light_scars','deformed_peduncle','deformed_counterpeduncle','bleaching']
segunda=['oil_spots','scars','light_scars','deformed_peduncle']
primera=['plu_stickers','green']

print(clases)
cols_tercera=[clases.index(x) for x in tercera]
cols_primera=[clases.index(x) for x in primera]
cols_segunda=[clases.index(x) for x in segunda]
print(cols_tercera)
print(cols_primera)
print(cols_segunda)

['cracks', 'damage_by_pliers', 'deformed_peduncle', 'green', 'hollow', 'insect', 'light_scars', 'mechanical_damage', 'oil_spots', 'plu_stickers', 'reddish', 'rotten', 'scars', 'skin_breakdown', 'surface']
[0, 11, 13, 7, 1, 14, 5, 4]
[9, 3]
[8, 12, 6, 2]


In [6]:
from matplotlib.backends.backend_pdf import PdfPages
from tqdm import tqdm
import sys
sys.path.append("../../src_clasificacion_vistas/common")
import pycimg

def crearMosaico(imagenes):
    """
    Crea un mosaico a partir de una lista de imágenes.
    """

    #Convertir las imágenes de tensores c x h x w float32 entre 0 y 1 a uint8 entre 0 y 255 h x w x c
    for i in range(len(imagenes)):
        imagenes[i] = (imagenes[i].permute(1, 2, 0).numpy() * 255).astype(np.uint8)
    
    # Obtener el tamaño de la imagen
    alto, ancho = imagenes[0].shape[:2]
    
    # Redimensionar todas las imágenes al tamaño de la primera
    # imagenes[0] = cv2.cvtColor(imagenes[0], cv2.COLOR_BGR2RGB)
    for i in range(1, len(imagenes)):
        imagenes[i] = cv2.resize(imagenes[i], (ancho, alto))
    #     imagenes[i] = cv2.cvtColor(imagenes[i], cv2.COLOR_BGR2RGB)

    # Calcular el número de filas y columnas
    num_imagenes = len(imagenes)
    num_columnas = int(np.ceil(np.sqrt(num_imagenes)))
    num_filas = int(np.ceil(num_imagenes / num_columnas))
    #print(f"num_imagenes: {num_imagenes}, num_columnas: {num_columnas}, num_filas: {num_filas}")

    # Crear una imagen en blanco para el mosaico
    mosaico = np.zeros((alto * num_filas, ancho * num_columnas, 3), dtype=np.uint8)

    # Colocar las imágenes en el mosaico
    for i, img in enumerate(imagenes):
        fila = i // num_columnas
        columna = i % num_columnas
        mosaico[fila * alto:(fila + 1) * alto, columna * ancho:(columna + 1) * ancho] = img

    return mosaico
def create_gold_master_pdf(data,clases,pdfname,elegidos):

    elegidos=[os.path.basename(x).replace(".npz","") for x in elegidos]

    with PdfPages(pdfname) as pdf:
        fig, axes=plt.subplots(1,2,figsize=(10,5),gridspec_kw={'width_ratios': [1, 2]})
        for d in tqdm(data):

            filenamenpz=d['filename']
            if not os.path.basename(filenamenpz).replace(".npz","") in elegidos:
                continue
            filenamejson=filenamenpz.replace(".npz",".json")

            
            scores=d['scores']
            ground_truth=d['ground_truth']
            gts=[v for k,v in ground_truth.items() if k in clases]
            preds=[v for k,v in scores.items() if k in clases]
            defectoss=[k for k,v in scores.items() if k in clases]
            
            kk=os.path.join("..",filenamejson)
            npzfilename=kk.replace(".json",".npz")


            imagenes=pycimg.npzread_torch(npzfilename,kk,channel_list=['R','G','B'])
            mosaico=crearMosaico(imagenes)
        #Generar imágenes de los errores

                    
            bn=os.path.basename(filenamejson)
            bn=bn.replace(".json","")        
            #fig, axes=plt.subplots(1,2,gridspec_kw={'width_ratios': [1, 2]})

            axes[0].bar(defectoss,gts)
            axes[0].bar(defectoss,preds)

            axes[0].tick_params(axis='x',rotation=90)
            axes[0].set_ylim([0,1])
            axes[0].set_yticks(np.arange(0, 1.1, 0.1))
            axes[0].grid()



            axes[1].imshow(mosaico)
            axes[1].axis('off')
            axes[1].set_title(f'{bn}')
            
            pdf.savefig(fig, bbox_inches='tight', dpi=300)
            
            axes[0].clear()
            axes[1].clear() 
    plt.close(fig)

In [7]:
def selecciona_columnas(preds,targets,cols):
    p=[]
    gt=[ ]
    for c in cols:
        p.append(preds[:,c])
        gt.append(targets[:,c])
    preds=torch.stack(p,dim=1)
    targets=torch.stack(gt,dim=1)
    preds=torch.max(preds,1)[0]
    targets=torch.max(targets,1)[0]
    return preds,targets

In [8]:
# Nos centramos en el conjunto de test

# Primero separamos por GT en primera segunda y tercera

umbral=0.5
preds_tercera,targets_tercera=selecciona_columnas(preds_test,labels_test,cols_tercera)
preds_primera,targets_primera=selecciona_columnas(preds_test,labels_test,cols_primera)
preds_segunda,targets_segunda=selecciona_columnas(preds_test,labels_test,cols_segunda)

print(targets_tercera.shape)
tercera_mask=targets_tercera>0.5
segunda_mask=(targets_segunda>0.5 )* torch.logical_not(tercera_mask)
primera_mask=torch.logical_not(tercera_mask)*torch.logical_not(segunda_mask)
print("Primera:",primera_mask.sum(), " ** Segunda:", segunda_mask.sum(), " **Tercera:",tercera_mask.sum())

confusion_matrix = torch.zeros(3, 3)

bueno_como_tercera=[]
tercera_como_bueno=[]
bueno_como_segunda=[]
for i in range(len(preds_tercera)):
    if tercera_mask[i]:
        categoria_real=3
    elif segunda_mask[i]:
        categoria_real=2
    elif primera_mask[i]:
        categoria_real=1

    if preds_tercera[i] > umbral:
        categoria_predicha=3
    elif preds_segunda[i] > umbral:
        categoria_predicha=2
    else:
        categoria_predicha=1
    confusion_matrix[categoria_real-1][categoria_predicha-1] += 1
    if categoria_real==1 and categoria_predicha==3:
        bueno_como_tercera.append(filenames_test[i])
    if categoria_real==1 and categoria_predicha==2:
        bueno_como_segunda.append(filenames_test[i])
    if categoria_real==3 and categoria_predicha==1:
        tercera_como_bueno.append(filenames_test[i])                     

print(confusion_matrix)
print("==========================================")
print("  ** BUENO COMO TERCERA **")
print(bueno_como_tercera)

print("==========================================")
print("  ** BUENO COMO SEGUNDA **")
print(bueno_como_segunda)
print("==========================================")
print("  ** TERCERA COMO BUENO **")
print(tercera_como_bueno)


torch.Size([162])
Primera: tensor(43)  ** Segunda: tensor(38)  **Tercera: tensor(81)
tensor([[38.,  2.,  3.],
        [ 6., 21., 11.],
        [ 4.,  4., 73.]])
  ** BUENO COMO TERCERA **
['data/mandarins/tango/KDC/20220805_180026_tangoPROD/20250218_1203_000691.npz', 'data/mandarins/nardocot/manfredi/20250224_0852_000007.npz', 'data/oranges/navel/manfredi2/deformed_counterpeduncle/20250403_0938_000028.npz']
  ** BUENO COMO SEGUNDA **
['data/mandarins/nardocot/manfredi/20250220_1239_000125.npz', 'data/oranges/navel/KDC/20220810_021928_Encoder_Buenas/20250218_1052_000420.npz']
  ** TERCERA COMO BUENO **
['data/mandarins/tango/KDC/20220804_230730_TANGO_encoder_good/20250218_1039_000407.npz', 'data/oranges/navel/KDC/20220829_214906_prod_naranja_plu/20250218_1211_000725.npz', 'data/oranges/navel/manfredi/20250224_0910_000139.npz', 'data/mandarins/clementine/manfredi2/light_scars/20250402_1321_002010.npz']


In [9]:

create_gold_master_pdf(test_data,clases,"bueno_como_tercera.pdf",bueno_como_tercera)

  0%|          | 0/162 [00:00<?, ?it/s]

100%|██████████| 162/162 [00:00<00:00, 337.47it/s]


In [10]:
create_gold_master_pdf(test_data,clases,"bueno_como_segunda.pdf",bueno_como_segunda)

100%|██████████| 162/162 [00:00<00:00, 545.02it/s]


In [11]:
create_gold_master_pdf(test_data,clases,"tercera_como_bueno.pdf",tercera_como_bueno)

100%|██████████| 162/162 [00:00<00:00, 169.83it/s]


# Umbrales distintos de 0.5

El umbral se selecciona en base a que para el producto de primera, no tenga defectos de 3ª o 2ª

In [29]:
# Nos centramos en el conjunto de test

# Primero separamos por GT en primera segunda y tercera

umbral=0.5
preds_tercera,targets_tercera=selecciona_columnas(preds_test,labels_test,cols_tercera)
preds_primera,targets_primera=selecciona_columnas(preds_test,labels_test,cols_primera)
preds_segunda,targets_segunda=selecciona_columnas(preds_test,labels_test,cols_segunda)

print(targets_tercera.shape)
tercera_mask=targets_tercera>0.5
segunda_mask=(targets_segunda>0.5 )* torch.logical_not(tercera_mask)
primera_mask=torch.logical_not(tercera_mask)*torch.logical_not(segunda_mask)
print("Primera:",primera_mask.sum(), " ** Segunda:", segunda_mask.sum(), " **Tercera:",tercera_mask.sum())


predicciones_producto_primera=preds_test[primera_mask]

umbrales=torch.max(predicciones_producto_primera,axis=0,keepdim=True)[0]+0.02

umbrales=torch.maximum(umbrales,torch.tensor(0.5))
print("Umbrales primera:",umbrales)
for clase,umbral in zip(clases,umbrales[0]):
    print(f'{clase} : {umbral:.2f}')

preds_test_binarizadas=preds_test>umbrales


preds_primera_binarizadas,targets_primera=selecciona_columnas(preds_test_binarizadas,labels_test,cols_primera)
preds_segunda_binarizadas,targets_segunda=selecciona_columnas(preds_test_binarizadas,labels_test,cols_segunda)
preds_tercera_binarizadas,targets_tercera=selecciona_columnas(preds_test_binarizadas,labels_test,cols_tercera)

confusion_matrix = torch.zeros(3, 3)

bueno_como_tercera=[]
tercera_como_bueno=[]
tercera_como_segunda=[]
segunda_como_tercera=[]
segunda_como_bueno=[]
bueno_como_segunda=[]
for i in range(len(preds_tercera)):
    if tercera_mask[i]:
        categoria_real=3
    elif segunda_mask[i]:
        categoria_real=2
    elif primera_mask[i]:
        categoria_real=1

    if preds_tercera_binarizadas[i] :
        categoria_predicha=3
    elif preds_segunda_binarizadas[i] :
        categoria_predicha=2
    else:
        categoria_predicha=1
    confusion_matrix[categoria_real-1][categoria_predicha-1] += 1
    if categoria_real==1 and categoria_predicha==3:
        bueno_como_tercera.append(filenames_test[i])
    if categoria_real==1 and categoria_predicha==2:
        bueno_como_segunda.append(filenames_test[i])
    if categoria_real==3 and categoria_predicha==2:
        tercera_como_segunda.append(filenames_test[i])   
    if categoria_real==3 and categoria_predicha==1:
        tercera_como_bueno.append(filenames_test[i])              
    if categoria_real==2 and categoria_predicha==1:
        segunda_como_bueno.append(filenames_test[i])    
    if categoria_real==2 and categoria_predicha==3:
        segunda_como_tercera.append(filenames_test[i])                

print(confusion_matrix)

print("==========================================")
print("  ** TERCERA COMO BUENO **")
print(tercera_como_bueno)

print("==========================================")
print("  ** TERCERA COMO SEGUNDA **")
print(tercera_como_segunda)


torch.Size([162])
Primera: tensor(43)  ** Segunda: tensor(38)  **Tercera: tensor(81)
Umbrales primera: tensor([[0.5000, 0.5000, 0.5000, 0.8020, 0.5470, 0.5000, 0.6120, 0.5000, 0.6780,
         0.9550, 0.9000, 0.5000, 0.5000, 0.5000, 0.5750]])
cracks : 0.50
damage_by_pliers : 0.50
deformed_peduncle : 0.50
green : 0.80
hollow : 0.55
insect : 0.50
light_scars : 0.61
mechanical_damage : 0.50
oil_spots : 0.68
plu_stickers : 0.95
reddish : 0.90
rotten : 0.50
scars : 0.50
skin_breakdown : 0.50
surface : 0.57
tensor([[43.,  0.,  0.],
        [ 9., 19., 10.],
        [ 7.,  6., 68.]])
  ** TERCERA COMO BUENO **
['data/mandarins/clementine/KDC/20220808_154231_Img_20220808_154156produccionClementina/20250218_1153_000571.npz', 'data/mandarins/tango/KDC/20220804_230730_TANGO_encoder_good/20250218_1039_000407.npz', 'data/oranges/navel/KDC/20220829_214906_prod_naranja_plu/20250218_1211_000725.npz', 'data/oranges/navel/manfredi/20250224_0910_000139.npz', 'data/mandarins/tango/KDC/20220805_180026_tango

In [30]:
create_gold_master_pdf(test_data,clases,"tercera_como_bueno_no_false_alarm.pdf",tercera_como_bueno)
create_gold_master_pdf(test_data,clases,"tercera_como_segunda_no_false_alarm.pdf",tercera_como_segunda)
create_gold_master_pdf(test_data,clases,"segunda_como_bueno_no_false_alarm.pdf",segunda_como_bueno)
create_gold_master_pdf(test_data,clases,"segunda_como_tercera_no_false_alarm.pdf",segunda_como_tercera)

100%|██████████| 162/162 [00:01<00:00, 111.28it/s]


# Elección de umbrales para evitar que se clasifique con severidad superior

Los umbrales se eligen para que:
* El umbral para los defectos de categoría 3 se escogen para que ningún producto de categoría 1 o 2 los supere

In [33]:
# Nos centramos en el conjunto de test

# Primero separamos por GT en primera segunda y tercera

umbral=0.5
preds_tercera,targets_tercera=selecciona_columnas(preds_test,labels_test,cols_tercera)
preds_primera,targets_primera=selecciona_columnas(preds_test,labels_test,cols_primera)
preds_segunda,targets_segunda=selecciona_columnas(preds_test,labels_test,cols_segunda)

print(targets_tercera.shape)
tercera_mask=targets_tercera>0.5
segunda_mask=(targets_segunda>0.5 )* torch.logical_not(tercera_mask)
primera_mask=torch.logical_not(tercera_mask)*torch.logical_not(segunda_mask)
primera_segunda_mask=torch.logical_not(tercera_mask)

print("Primera:",primera_mask.sum(), " ** Segunda:", segunda_mask.sum(), " **Tercera:",tercera_mask.sum())


predicciones_producto_primera=preds_test[primera_mask]
predicciones_producto_primera_segunda=preds_test[primera_segunda_mask]

umbrales=torch.max(predicciones_producto_primera,axis=0,keepdim=True)[0]+0.02
umbrales=torch.maximum(umbrales,torch.tensor(0.5))

umbrales2=torch.max(predicciones_producto_primera_segunda,axis=0,keepdim=True)[0]+0.02
umbrales2=torch.maximum(umbrales,torch.tensor(0.5))
print("Umbrales primera:",umbrales)
for clase,umbral in zip(clases,umbrales[0]):
    print(f'{clase} : {umbral:.2f}')

umbrales_globales=[]
for clase,umbral,umbral2 in zip(clases,umbrales[0],umbrales2[0]):
    if clase in tercera:
        umbrales_globales.append(umbral2)
    else:
        umbrales_globales.append(umbral)
        
print("Umbrales Globales:",umbrales)
for clase,umbral in zip(clases,umbrales_globales):
    print(f'{clase} : {umbral:.2f}')

preds_test_binarizadas=preds_test>umbrales


preds_primera_binarizadas,targets_primera=selecciona_columnas(preds_test_binarizadas,labels_test,cols_primera)
preds_segunda_binarizadas,targets_segunda=selecciona_columnas(preds_test_binarizadas,labels_test,cols_segunda)
preds_tercera_binarizadas,targets_tercera=selecciona_columnas(preds_test_binarizadas,labels_test,cols_tercera)

confusion_matrix = torch.zeros(3, 3)

bueno_como_tercera=[]
tercera_como_bueno=[]
tercera_como_segunda=[]
segunda_como_tercera=[]
segunda_como_bueno=[]
bueno_como_segunda=[]
for i in range(len(preds_tercera)):
    if tercera_mask[i]:
        categoria_real=3
    elif segunda_mask[i]:
        categoria_real=2
    elif primera_mask[i]:
        categoria_real=1

    if preds_tercera_binarizadas[i] :
        categoria_predicha=3
    elif preds_segunda_binarizadas[i] :
        categoria_predicha=2
    else:
        categoria_predicha=1
    confusion_matrix[categoria_real-1][categoria_predicha-1] += 1
    if categoria_real==1 and categoria_predicha==3:
        bueno_como_tercera.append(filenames_test[i])
    if categoria_real==1 and categoria_predicha==2:
        bueno_como_segunda.append(filenames_test[i])
    if categoria_real==3 and categoria_predicha==2:
        tercera_como_segunda.append(filenames_test[i])   
    if categoria_real==3 and categoria_predicha==1:
        tercera_como_bueno.append(filenames_test[i])              
    if categoria_real==2 and categoria_predicha==1:
        segunda_como_bueno.append(filenames_test[i])    
    if categoria_real==2 and categoria_predicha==3:
        segunda_como_tercera.append(filenames_test[i])                

print(confusion_matrix)

print("==========================================")
print("  ** TERCERA COMO BUENO **")
print(tercera_como_bueno)

print("==========================================")
print("  ** TERCERA COMO SEGUNDA **")
print(tercera_como_segunda)


torch.Size([162])
Primera: tensor(43)  ** Segunda: tensor(38)  **Tercera: tensor(81)
Umbrales primera: tensor([[0.5000, 0.5000, 0.5000, 0.8020, 0.5470, 0.5000, 0.6120, 0.5000, 0.6780,
         0.9550, 0.9000, 0.5000, 0.5000, 0.5000, 0.5750]])
cracks : 0.50
damage_by_pliers : 0.50
deformed_peduncle : 0.50
green : 0.80
hollow : 0.55
insect : 0.50
light_scars : 0.61
mechanical_damage : 0.50
oil_spots : 0.68
plu_stickers : 0.95
reddish : 0.90
rotten : 0.50
scars : 0.50
skin_breakdown : 0.50
surface : 0.57
Umbrales Globales: tensor([[0.5000, 0.5000, 0.5000, 0.8020, 0.5470, 0.5000, 0.6120, 0.5000, 0.6780,
         0.9550, 0.9000, 0.5000, 0.5000, 0.5000, 0.5750]])
cracks : 0.50
damage_by_pliers : 0.50
deformed_peduncle : 0.50
green : 0.80
hollow : 0.55
insect : 0.50
light_scars : 0.61
mechanical_damage : 0.50
oil_spots : 0.68
plu_stickers : 0.95
reddish : 0.90
rotten : 0.50
scars : 0.50
skin_breakdown : 0.50
surface : 0.57
tensor([[43.,  0.,  0.],
        [ 9., 19., 10.],
        [ 7.,  6., 6